In [1]:
!git clone https://github.com/cristinalunaj/MMEmotionRecognition.git

Cloning into 'MMEmotionRecognition'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (323/323), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 328 (delta 82), reused 303 (delta 66), pack-reused 5
Receiving objects: 100% (328/328), 24.56 MiB | 10.83 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [2]:
%%capture
!pip install datasets
!pip install transformers wandb 


In [3]:
%cd MMEmotionRecognition/


/content/MMEmotionRecognition


In [4]:
%load_ext autoreload
%autoreload 2

In [6]:
%%capture
!wget https://zenodo.org/record/1188976/files/Video_Speech_Actor_01.zip?download=1

In [ ]:

!unzip /content/MMEmotionRecognition/Video_Speech_Actor_01.zip?download=1

Archive:  /content/MMEmotionRecognition/Video_Speech_Actor_01.zip?download=1
replace Actor_01/01-01-01-01-01-01-01.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Actor_01/01-01-01-01-01-01-01.mp4  
replace Actor_01/01-01-01-01-01-02-01.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
import argparse


##Extract audio from video

In [ ]:

parser = argparse.ArgumentParser(description="Configuration of setup and training process")
parser.add_argument('-videos', '--videos_dir', type=str, default="/content/MMEmotionRecognition/Actor_01",
                    help='Path with the videos to extract the audios')
parser.add_argument('-out', '--out_dir', type=str, help='Path to save the processed audios at 16kHzs and mono channel',
                    default='./audio_16khz')

args = parser.parse_args(args=[])
args

Namespace(out_dir='./audio_16khz', videos_dir='/content/MMEmotionRecognition/Actor_01')

In [ ]:

import os
from src.Audio.preProcessing.process_audio import convert_audio216k
os.makedirs(args.out_dir, exist_ok=True)
for video_i in os.listdir(args.videos_dir):
    path_video = os.path.join(args.videos_dir, video_i)
    path_out_audio = os.path.join(args.out_dir, video_i.rsplit(".")[0]+".wav")
    print("Processing ", path_video)
    convert_audio216k(path_video, path_out_audio)

Processing  /content/MMEmotionRecognition/Actor_01/01-01-04-02-02-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/02-01-05-01-02-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-05-01-01-01-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-01-01-01-01-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-04-01-02-01-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-08-01-02-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-06-02-01-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-06-01-01-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/02-01-03-01-01-01-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/02-01-03-02-01-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/02-01-06-01-02-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/02-01-03-02-02-02-01.mp4
Processing  /content/MMEmotionRecognition/Actor_01/01-01-07-02-01-02-01.mp4
Processing  

##Fine Tunning 

In [ ]:
parser = argparse.ArgumentParser(description="Configuration of setup and training process")
parser.add_argument('-audios', '--audios_dir', type=str,default="/content/MMEmotionRecognition/audio_16khz",
                    help='Path with the audios to train/test the model')
parser.add_argument('-cache', '--cache_dir', type=str,default="./",
                    help='Path to save aux. data in cache')
parser.add_argument('-out', '--out_dir', type=str, help='Path to save the embeddings extracted from the model',
                    default='./fineTuneWav2vec')
parser.add_argument('-model', '--model_id', type=str,
                    help='Model identificator in Hugging Face library [default: jonatasgrosman/wav2vec2-large-xlsr-53-english]',
                    default='jonatasgrosman/wav2vec2-large-xlsr-53-english')

args = parser.parse_args(args=[])
args

Namespace(audios_dir='/content/MMEmotionRecognition/audio_16khz', cache_dir='./', model_id='jonatasgrosman/wav2vec2-large-xlsr-53-english', out_dir='./fineTuneWav2vec')

In [ ]:
!rm -r /content/MMEmotionRecognition/fineTuneWav2vec/trained_models

rm: cannot remove '/content/MMEmotionRecognition/fineTuneWav2vec/trained_models': No such file or directory


In [ ]:
import wandb 
wandb.init(project="MMErecognition")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import os
import sys
import argparse
sys.path.append('.')
sys.path.append('..')
sys.path.append('../../')
sys.path.append('../../../')

os.environ['LC_ALL'] ='C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import random
import numpy as np
import pandas as pd
import time

from pathlib import Path
from tqdm import tqdm

import torchaudio

#PREPARE DATA
from datasets import load_dataset, load_metric
#LOAD PRE-TRAINED MODEL ON ASR
from transformers import AutoConfig

from src.Audio.FineTuningWav2Vec.DataCollatorCTCWithPadding import *
from src.Audio.FineTuningWav2Vec.Wav2VecAuxClasses import *
from src.Audio.FineTuningWav2Vec.CTCTrainer import *

#TRAINING
from transformers import EvalPrediction
from transformers import TrainingArguments
from datetime import datetime

from datasets import set_caching_enabled
set_caching_enabled(False)
from sklearn.model_selection import train_test_split

from src.Audio.FineTuningWav2Vec.main_FineTuneWav2Vec_CV import prepare_RAVDESS_DS,seed_torch,generate_train_test,seed_libs,speech_file_to_array_fn,label_to_id,compute_metrics


def generate_train_test(fold, df, save_path=""):
    """
    Divide the data in train and test in a subject-wise 5-CV way. The division is generated before running the training
    of each fold.

    :param fold:[int] Fold to create the train and test sets [ranging from 0 - 4]
    :param df:[DataFrame] Dataframe with the complete list of files generated by prepare_RAVDESS_DS(..) function
    :param save_path:[str] Path to save the train.csv and test.csv per fold
    """
    actors_per_fold = {
        0: [2,5,14,15,16],
        1: [3, 6, 7, 13, 18],
        2: [10, 11, 12, 19, 20],
        3: [8, 17, 21, 23, 24],
        4: [1, 4, 9, 22],
    }

    train_df, test_df = train_test_split(df, test_size=0.2)

    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    if(save_path!=""):
        train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
        test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)
    return train_df, test_df
out_dir_models = os.path.join(args.out_dir, "trained_models/wav2vec2-xlsr-ravdess-speech-emotion-recognition") #out path to save trained models
data_path = os.path.join(args.out_dir,"data") #Path to save csvs generated containing the recording information (path, name, emotion...)
# We need to specify the input and output column
input_column = "path" # Name of the column that will contain the path of the recordings
output_column = "emotion" # Name of the column that will contain the labels of the recordings
pooling_mode = "mean" #Type of pooling to apply to the embeddings generated ath the output of the transformer module to collapse all the timesteps of the recordingsinto a single vector
now = datetime.now()
current_time = now.strftime("%Y%m%d_%H%M%S")
seed = 2020
epochs = 1 #Epochs to train the model
#PARAMETERS #######################
seed_torch(seed=seed) #Set random seeds


for fold in range(1):

  #Define paths, create aux. folders and callbacks to save data
  out_dir_models_path = os.path.join(out_dir_models, current_time, "fold"+str(fold))
  save_path = os.path.join(data_path, current_time, "fold"+str(fold))
  os.environ['TRANSFORMERS_CACHE'] = os.path.join(args.cache_dir, current_time, "fold"+str(fold))
  os.environ['HF_DATASETS_CACHE'] = os.path.join(args.cache_dir, current_time, "fold"+str(fold))
  os.makedirs(save_path, exist_ok=True)
  print("SAVING DATA IN: ", save_path)
  callbackTB = transformers.integrations.TensorBoardCallback()
  #######################

  #PREPARE DATASET
  #Generate complete dataframe with RAVDESS samples
  df = prepare_RAVDESS_DS(args.audios_dir)
  print("##################")
  print(df)
  print("##################")
  _, _ = generate_train_test(fold, df, save_path)
  time.sleep(10) #wait some time to get the dataset ready


  data_files = {
      "train": os.path.join(save_path, "train.csv"),
      "validation": os.path.join(save_path, "test.csv"),
  }
  #Load data
  dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
  train_dataset = dataset["train"]
  eval_dataset = dataset["validation"]
  print("Processing fold: ", str(fold), " - actors in Train fold: ",set(train_dataset["actor"]))
  print("Processing fold: ", str(fold), " - actors in Eval fold: ", set(eval_dataset["actor"]))
  label_list = train_dataset.unique(output_column)
  label_list.sort()  # Let's sort it for determinism
  num_labels = len(label_list)
  print(f"A classification problem with {num_labels} classes: {label_list}")
  # LOAD PRE-TRAINED MODEL ON ASR
  # config
  config = AutoConfig.from_pretrained(
      args.model_id, #path to the model of HuggingFace lib. that we will use as baseline to fine-tune.
      num_labels=num_labels, # num classes
      label2id={label: i for i, label in enumerate(label_list)}, # dict that maps emotions -> numbers
      id2label={i: label for i, label in enumerate(label_list)}, # dict that maps numbers -> emotions
      finetuning_task="wav2vec2_clf",
  )
  #Add in the config variable the 'pooling_mode'
  setattr(config, 'pooling_mode', pooling_mode)
  #Load the processor for the type of model (Wav2Vec2.0 in our case) and get the expected sampling rate (16kHZ in our case)
  processor = Wav2Vec2Processor.from_pretrained(args.model_id, )
  target_sampling_rate = processor.feature_extractor.sampling_rate
  print(f"The target sampling rate: {target_sampling_rate}")
  def speech_file_to_array_fn(path):
    """
    Loader of audio recordings. It loads the recordings and convert them to a specific sampling rate if required, and returns
    an array with the samples of the audio.

    :param path:[str] Path to the wav file.
    :param target_sampling_rate:[int] Global variable with the expected sampling rate of the model
    """
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech
  def preprocess_function(examples, input_column = "path", output_column = "emotion"):
    """
    Load the recordings with their labels.

    :param examples:[DataFrame]  with the samples of the training or test sets.
    :param input_column:[str]  Column that contain the paths to the recordings
    :param output_column:[str]  Column that contain the emotion associated to each recording
    :param target_sampling_rate:[int] Global variable with the expected sampling rate of the model
    """
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result
  print("Generating training...")
  train_dataset = train_dataset.map(
      preprocess_function,
      batch_size=100,
      batched=True,
      num_proc=4
  )
  print("Generating test...")
  eval_dataset = eval_dataset.map(
      preprocess_function,
      batch_size=100,
      batched=True,
      num_proc=4
  )
  #MODEL
  print("Training model...")
  data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
  is_regression = False

  #Create the architecture: Wav2Vec2.0 model + mean pooling + MLP (1024, 8)
  model = Wav2Vec2ForSpeechClassification.from_pretrained(
      args.model_id,
      config=config,
  )

  #Freeze feature encoder layers (CNNs) of wav2vec2.0 & train the transformer module and the MLP that we have added (randomly initialized)
  model.freeze_feature_extractor()

  #Set trainig arguments/parameters
  training_args = TrainingArguments(
      output_dir=out_dir_models_path,
      per_device_train_batch_size=2,
      per_device_eval_batch_size=2,
      gradient_accumulation_steps=2,
      evaluation_strategy="steps",
      prediction_loss_only=False,
      num_train_epochs=epochs,
      fp16=True,
      save_steps=10,
      eval_steps=10,
      logging_steps=10,
      learning_rate=1e-4,
      save_total_limit=1,
      load_best_model_at_end=True,
      metric_for_best_model="eval_accuracy",
      seed=seed, )
  #Set data collator to pad the small recordings
  def compute_metrics(p: EvalPrediction):
        """
        Extract the metrics of the model from the predictions.
            -MSE for regression tasks
            -Accuracy for classification tasks
        :param p: EvalPrediction: Predictions of the model.
      """
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

        if is_regression:
            return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
        else:
            return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}
  trainer = CTCTrainer(
      model=model,
      data_collator=data_collator,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      tokenizer=processor.feature_extractor,
      callbacks = [callbackTB])

  #Start training the network using the train_dataset & evaluating it on the eval_dataset passed as parameters
  # to the CTCTrainer

  trainer.train()

SAVING DATA IN:  ./fineTuneWav2vec/data/20220311_223027/fold0


120it [00:00, 29265.99it/s]

##################
                     name                                               path  \
0    02-01-06-02-02-01-01  /content/MMEmotionRecognition/audio_16khz/02-0...   
1    01-01-05-01-01-02-01  /content/MMEmotionRecognition/audio_16khz/01-0...   
2    02-01-06-01-01-02-01  /content/MMEmotionRecognition/audio_16khz/02-0...   
3    01-01-01-01-01-01-01  /content/MMEmotionRecognition/audio_16khz/01-0...   
4    02-01-06-01-02-01-01  /content/MMEmotionRecognition/audio_16khz/02-0...   
..                    ...                                                ...   
115  01-01-07-02-02-01-01  /content/MMEmotionRecognition/audio_16khz/01-0...   
116  01-01-06-01-01-01-01  /content/MMEmotionRecognition/audio_16khz/01-0...   
117  02-01-06-02-01-01-01  /content/MMEmotionRecognition/audio_16khz/02-0...   
118  01-01-07-02-02-02-01  /content/MMEmotionRecognition/audio_16khz/01-0...   
119  01-01-03-01-02-01-01  /content/MMEmotionRecognition/audio_16khz/01-0...   

     emotion  actor 


Using custom data configuration default-343fe5bf5b6f4352


  0%|          | 0/2 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this mode

Step,Training Loss,Validation Loss,Accuracy
10,2.165700,2.096924,0.083333


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: name, path, actor, emotion. If name, path, actor, emotion are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 24
  Batch size = 2
Saving model checkpoint to ./fineTuneWav2vec/trained_models/wav2vec2-xlsr-ravdess-speech-emotion-recognition/20220311_223027/fold0/checkpoint-10
Configuration saved in ./fineTuneWav2vec/trained_models/wav2vec2-xlsr-ravdess-speech-emotion-recognition/20220311_223027/fold0/checkpoint-10/config.json
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 379, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 499, in _save
    zip_file.write_record(name, storage.data_ptr(), num_bytes)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-b074deb72f17>", line 236, in <module>
    trainer.train()
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1475, in train
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1606, in _maybe_log_save_evaluate
 

TypeError: ignored

##Evaluation

In [ ]:
parser = argparse.ArgumentParser(description="Configuration of setup and training process")
parser.add_argument('-d', '--data', type=str, default=save_path[:-6],
                    help='Path with the datasets automatically generated with the Fine-Tuning script (train.csv and test.csv)')
parser.add_argument('-trainedModel', '--trained_model', type=str, default=out_dir_models_path[:-6] ,
                    help='Path to the trained model')
parser.add_argument('-fold', '--fold', type=int, help='Fold number',
                    default=0)
parser.add_argument('-out', '--out_dir', type=str, help='Path to save the embeddings extracted from the model',
                    default='./wav2vec_emb')
parser.add_argument('-model', '--model_id', type=str,
                    help='Model identificator in Hugging Face library [default: jonatasgrosman/wav2vec2-large-xlsr-53-english]',
                    default='jonatasgrosman/wav2vec2-large-xlsr-53-english')
args = parser.parse_args(args=[])
args

In [ ]:
import sys
import argparse
sys.path.append('.')
sys.path.append('..')
sys.path.append('../../')
sys.path.append('../../../')
import os.path

import librosa
import pandas as pd
from sklearn.metrics import classification_report
from datasets import load_dataset
from transformers import AutoConfig, Wav2Vec2Processor
import torchaudio
import numpy as np
import torch
from src.Audio.FineTuningWav2Vec.Wav2VecAuxClasses import *
from src.Audio.FineTuningWav2Vec.Wav2VecEval import *
#Change if you change the model to choose top weigths
checkpoints_per_fold = {0: "checkpoint-10",
                        1: "checkpoint-20",
                        }

trained_model_name = args.trained_model.split("/")[-1]
path_out_logits = os.path.join(args.out_dir, trained_model_name, "fold"+str(args.fold))
os.makedirs(path_out_logits, exist_ok=True)

for set_trainTest in ["train","test"]:
    #Load the data from train.csv or test.csv
    test_dataset = load_dataset("csv", data_files={"test": os.path.join(args.data, "fold"+str(args.fold), set_trainTest+".csv")}, delimiter="\t")["test"]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #Load the fine-tuned model and its associated processor that has information about the input/output format of the files/outouts in the trianed model
    model_train_val_test = os.path.join(args.trained_model, "fold"+str(args.fold), checkpoints_per_fold[args.fold])
    config = AutoConfig.from_pretrained(os.path.join(model_train_val_test, 'config.json'))
    processor = Wav2Vec2Processor.from_pretrained(args.model_id)
    model = Wav2Vec2ForSpeechClassification.from_pretrained(model_train_val_test).to(device)
    def predict(batch):
      """
      Generate predictions from the model and append to the batch dict the posteriors and predictions

      :param batch:[dict] Dict with the data
                              -speech: input audio recordings [IN]
                              -posteriors: Array with the weight of each of the ourput neuron assigned to each class [OUT]
                              -predicted : Predicted class with the highest posterior [OUT]
      :param: device [str]: Global variable with the device used to load the model and make predictions ('cpu' or 'cuda')

      """
      features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

      input_values = features.input_values.to(device)
      attention_mask = features.attention_mask.to(device)

      with torch.no_grad():
          logits = model(input_values, attention_mask=attention_mask).logits

      pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
      batch["posteriors"] = logits.detach().cpu().numpy()
      batch["predicted"] = pred_ids
      return batch

    def speech_file_to_array_fn(batch):
        """
        Loader of audio recordings. It appends the array of the samples of the recording to the batch dict

        :param batch:[dict] Dict with the data
        :param processor: Global variable with the expected input format of the model
        """
        speech_array, sampling_rate = torchaudio.load(batch["path"])
        speech_array = speech_array.squeeze().numpy()
        speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

        batch["speech"] = speech_array
        return batch

    #Load the audio samples
    test_dataset = test_dataset.map(speech_file_to_array_fn)
    # Make predictions on the loaded samples
    result = test_dataset.map(predict, batched=True, batch_size=8)
    label2id_dict = {"Neutral": 0, "Calm": 1, "Happy": 2, "Sad": 3, "Angry": 4, "Fear": 5, "Disgust": 6, "Surprise": 7}

    print("LABELS: ", config.label2id)
    # Get the labels and predictions
    y_true = [config.label2id[name] for name in result["emotion"]]
    y_pred = result["predicted"]

    #Get metrics & save posteriors
    print(classification_report(y_true, y_pred, target_names=list(label2id_dict)))
    #Save embs:
    out_embs = os.path.join(path_out_logits, "posteriors_"+set_trainTest+".csv")
    def save_embs(results, path_embs):
      """
      Generate predictions from the model and append to the batch dict the posteriors and predictions

      :param batch:[dict] Dict with the data
                              -speech: input audio recordings [IN]
                              -posteriors: Array with the weight of each of the ourput neuron assigned to each class [OUT]
                              -predicted : Predicted class with the highest posterior [OUT]

      """
      df_posteriors = pd.DataFrame([])
      df_posteriors[["embs"+str(i) for i in range(8)]] = results["posteriors"]
      df_posteriors["name"] = result["name"]
      df_posteriors["emotion"] = result["emotion"]
      df_posteriors["actor"] = result["actor"]
      #Save dataframe
      df_posteriors.to_csv(path_embs, sep=";", header=True, index=False)


    save_embs(result, out_embs)

##Feature extraction wav2vec

In [ ]:
import argparse

parser = argparse.ArgumentParser(description="Configuration of setup and training process")
parser.add_argument('-d', '--data', type=str,default=save_path,
                    help='Path with the datasets automatically generated with the Fine-Tuning script (train.csv and test.csv)')
parser.add_argument('-out', '--out_dir', type=str, help='Path to save the embeddings extracted from the model', default='./output')
parser.add_argument('-model', '--model_id', type=str, help='Model identificator in Hugging Face library [default: jonatasgrosman/wav2vec2-large-xlsr-53-english]',
                    default='jonatasgrosman/wav2vec2-large-xlsr-53-english')

args = parser.parse_args(args=[])
args


In [ ]:
!mkdir output
from src.Audio.FeatureExtractionWav2Vec.FeatureExtractor import load_dataset, extract_features, speech_file_to_array_fn
import argparse
import os, sys
sys.path.append('.')
sys.path.append('..')
sys.path.append('../../')
sys.path.append('../../../')
import librosa
import pandas as pd
from datasets import load_dataset
from transformers import Wav2Vec2Processor
import torchaudio
import numpy as np
import torch
from src.Audio.FineTuningWav2Vec.Wav2VecAuxClasses import Wav2Vec2Model 

def speech_file_to_array_fn(batch):
    """
        Loader of audio recordings. It appends the array of the samples of the recording to the batch dict

        :param batch:[dict] Dict with the data
        :param processor[Wav2Vec2Processor]: Global variable with the expected input format of the model
        """
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch

test_dataset = load_dataset("csv", data_files={"test": os.path.join(args.data, "train.csv")}, delimiter="\t")["test"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2Processor.from_pretrained(args.model_id)
model = Wav2Vec2Model.from_pretrained(args.model_id).to(device)
test_dataset = test_dataset.map(speech_file_to_array_fn)
cols = ["embs" + str(i) for i in range(512)]
for row in test_dataset:
    result = extract_features(row, device, model, processor)
    df_aux = pd.DataFrame(result['predicted'].cpu().numpy().reshape(-1, 512), columns=cols)
    df_aux.to_csv(os.path.join(args.out_dir, result["name"]+".csv"), sep=";", index=False, header=True)
